In [2]:
from src.env_loader import load_api_key
import giskard
import os
from giskard.llm.client.openai import OpenAIClient

load_api_key()

MODEL = "gpt-3.5-turbo"

# Set up Giskard to use gpt-3.5-turbo instead of the default gpt-4
giskard.llm.set_llm_api("openai")
oc = OpenAIClient(model=MODEL)
giskard.llm.set_default_client(oc)

/home/hilla/code/10Academy-training/week7/Prompt-Tuning-Enterprise-RAG/.venv/lib/python3.9/site-packages/giskard/core/validation.py:31: RuntimeWarning: fields may not start with an underscore, ignoring "_args"
  validated_func = validate_arguments(func, config={"arbitrary_types_allowed": True})
/home/hilla/code/10Academy-training/week7/Prompt-Tuning-Enterprise-RAG/.venv/lib/python3.9/site-packages/giskard/core/validation.py:31: RuntimeWarning: fields may not start with an underscore, ignoring "_kwargs"
  validated_func = validate_arguments(func, config={"arbitrary_types_allowed": True})


###  Scrape the Website and Split the Content


In [3]:
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)

loader = WebBaseLoader("https://www.ml.school/")
documents = loader.load_and_split(text_splitter)
documents

[Document(page_content='Building Machine Learning Systems That Don\'t Suck"This is the best machine learning course I\'ve done. Worth every cent."Jose Reyes, AI/ML at Cevo AustraliaBuilding Machine Learning Systems That Don\'t SuckA live, interactive program that\'ll help you build production-ready machine learning systems from the ground up.Next cohort:\xa0July 1 - 18, 2024Check the schedule for more details about upcoming cohorts.I want to join!Sign inLearn how to design, build, deploy, and scale machine learning systems to solve real-world problems.I\'ll lose my mind if I see another book or course teaching people the same basic ideas for the hundredth time. Most people are stuck in beginner mode, and finding help to solve real-world problems is hard.I want to change that.I started writing software 30 years ago. I\'ve written pipelines and trained models for some of the largest companies in the world. I want to show you how to do the same.This is the class I wish I had taken when I 

#### Load the Content in a Vector Store


In [4]:
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(
    documents, embedding=OpenAIEmbeddings()
)

#### Create a Knowledge Base


In [5]:
import pandas as pd

df = pd.DataFrame([d.page_content for d in documents], columns=["text"])
df.head(10)

,text
0,Building Machine Learning Systems That Don't S...
1,program will help you unlearn what you think m...
2,only pay once to join. There are no monthly fe...
3,that make systems work.You are ready to put in...
4,"testing in production, among many others.You'l..."
5,every Monday and Thursday at the same time. On...
6,can participate in as many iterations as you'd...
7,collection strategies. A technique to determin...
8,and I continue learning. I recommend it with c...
9,many fundamental ideas behind machine learning...


In [6]:
from giskard.rag import KnowledgeBase

knowledge_base = KnowledgeBase(df)

#### Generate the Test Set


In [7]:
from giskard.rag import generate_testset

testset = generate_testset(
    knowledge_base,
    num_questions=60,
    agent_description="A chatbot answering questions about the Machine Learning School Website",
)

2024-06-06 08:12:30,874 pid:488241 MainThread giskard.rag  INFO     Finding topics in the knowledge base.


/home/hilla/code/10Academy-training/week7/Prompt-Tuning-Enterprise-RAG/.venv/lib/python3.9/site-packages/umap/umap_.py:2437: UserWarning: n_neighbors is larger than the dataset size; truncating to X.shape[0] - 1
  warn(


2024-06-06 08:12:45,281 pid:488241 MainThread giskard.rag  INFO     Found 1 topics in the knowledge base.


Generating questions:   0%|          | 0/60 [00:00<?, ?it/s]

Display a few samples in the test test

In [8]:
test_set_df = testset.to_pandas()

for index, row in enumerate(test_set_df.head(3).iterrows()):
    print(f"Question {index + 1}: {row[1]['question']}")
    print(f"Reference answer: {row[1]['reference_answer']}")
    print("Reference context:")
    print(row[1]['reference_context'])
    print("******************", end="\n\n")

Question 1: When was the program launched?
Reference answer: The program was launched in March 2023.
Reference context:
Document 10: this program in March 2023. Since then, thousands of students have graduated, and I can't wait to meet you in class.Copyright © 2024 Tideily LLCAll rights reserved.
******************

Question 2: What does 'lifetime access' mean in the program?
Reference answer: 'Lifetime access' means that you only pay once to join the program and get immediate access to every past, present, and future cohort. This includes all updates and improvements made in each iteration of the program.
Reference context:
Document 1: program will help you unlearn what you think machine learning is. It's a practical, hands-on class where you'll learn from years of experience and real-world examples.When you join, you get lifetime access to the following:18 hours of live, interactive sessions. We'll use this time to discuss the first principles behind building machine learning systems

Save the test set to a file

In [9]:
testset.save("test-set.jsonl")

#### Prepare the prompt template

In [10]:
from langchain.prompts import PromptTemplate

template = """
Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
print(prompt.format(context="Here is some context", question="Here is a question"))


Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: Here is some context

Question: Here is a question



#### Create a RAG chain

Create a retriever from the Vector Store that will allow us to get the top similar documents to a given question.



In [11]:

retriever = vectorstore.as_retriever()
retriever.get_relevant_documents("What is the Machine Learning School?")

/home/hilla/code/10Academy-training/week7/Prompt-Tuning-Enterprise-RAG/.venv/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


[Document(page_content="program will help you unlearn what you think machine learning is. It's a practical, hands-on class where you'll learn from years of experience and real-world examples.When you join, you get lifetime access to the following:18 hours of live, interactive sessions. We'll use this time to discuss the first principles behind building machine learning systems.10 hours of step-by-step coding instructions. These practical sessions will show you how to build an end-to-end system from scratch.A final project where you'll build a complete solution and receive direct feedback on your work.100 coding assignments and practice questions.The entire source code of a working production system. It's yours. You can change and use it as you see fit.A private community where you'll collaborate with thousands of people from different backgrounds.Direct access to your instructor.Lifetime access to every past and future cohort.Program certificate upon completion.And the best part is tha

##### Create the chain

In [12]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter

model = ChatOpenAI(model=MODEL)

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
    }
    | prompt
    | model
    | StrOutputParser()
)

Make sure the chain works

In [13]:
chain.invoke({"question": "What is the Machine Learning School?"})


'The Machine Learning School is a live, interactive program that helps individuals build production-ready machine learning systems from the ground up.'

### Evaluate the MOdel on the Test set

In [14]:
def answer_fn(question, history=None):
    return chain.invoke({"question": question})


We can now use the evaluate() function to evaluate the model on the test set. This function will compare the answers from the chain with the reference answers in the test set.

In [15]:
from giskard.rag import evaluate

report = evaluate(answer_fn, testset=testset, knowledge_base=knowledge_base)

Asking questions to the agent:   0%|          | 0/60 [00:00<?, ?it/s]

CorrectnessMetric evaluation:   0%|          | 0/60 [00:00<?, ?it/s]

Let now display the report.

Here are the five components of our RAG application:

Generator: This is the LLM used in the chain to generate the answers.
Retriever: This is the retriever that fetches relevant documents from the knowledge base according to a query.
Rewriter: This is a component that rewrites the user query to make it more relevant to the knowledge base or to account for chat history.
Router: This is a component that filters the query of the user based on his intentions.
Knowledge Base: This is the set of documents given to the RAG to generate the answers.

In [16]:
display(report)


Loading BokehJS ...